<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Wikipedia_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streaming Wikipedia Events

FROM: https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf



In [ ]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Wikipedia Event") \
    .getOrCreate()

### Streaming Wikipedia events

Currently, Spark supports three kinds of streaming connection out of the box:

FROM:
https://github.com/donaghhorgan/COMP9033/blob/master/lectures/12%20-%20Streaming%20Data%20Analysis.pdf 

1. Connect to the Wikipedia RecentChanges stream using SSEClient.
2. Create a local socket connection on port 50000.
3. When a client (e.g. Spark) connects to the local socket, relay the next available event to it from the event stream.

In [ ]:
!pip install sseclient
from pyspark.streaming import StreamingContext
from sseclient import SSEClient
import threading

In [ ]:
events = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')
for e in events:
  print(e)

In [ ]:
def relay():
    events = SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')
    
    s = socket.socket()
    s.bind(('localhost', 50000))
    s.listen(1)
    while True:
        try:
            client, address = s.accept()
            for event in events:
                if event.event == 'message':
                    client.sendall(event.data)
                    break
        except:
            pass
        finally:
            client.close()
    

threading.Thread(target=relay).start()
sse=SSEClient('https://stream.wikimedia.org/v2/stream/recentchange')
sse.next

## Streaming analysis

Now that we have our stream relay set up, we can start to analyse its contents. First, let's initialise a [`SparkContext`](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.SparkContext) object, which will represent our connection to the Spark cluster. To do this, we must first specify the URL of the master node to connect to. As we're only running this notebook for demonstration purposes, we can just run the cluster locally, as follows:

Next, we create a [`StreamingContext`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext) object, which represents the streaming functionality of our Spark cluster. When we create the context, we must specify a batch duration time (in seconds), to tell Spark how often it should process data from the stream. Let's process the Wikipedia data in batches of one second:

In [ ]:
ssc = StreamingContext(sc, 1)

Using our `StreamingContext` object, we can create a data stream from our local TCP relay socket with the [`socketTextStream`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.socketTextStream) method:

In [ ]:
stream = ssc.socketTextStream('localhost', 50000)

Even though we've created a data stream, nothing happens! Before Spark starts to consume the stream, we must first define one or more operations to perform on it. Let's count the number of edits made by different users in the last minute:

In [ ]:
users = (
    stream.map(json.loads)                   # Parse the stream data as JSON
          .map(lambda obj: obj['user'])      # Extract the values corresponding to the 'user' key
          .map(lambda user: (user, 1))       # Give each user a count of one
          .window(60)                        # Create a sliding window, sixty seconds in length
          .reduceByKey(lambda a, b: a + b)   # Reduce all key-value pairs in the window by adding values
          .transform(                        # Sort by the largest count
              lambda rdd: rdd.sortBy(lambda kv: kv[1], ascending=False))
          .pprint()                          # Print the results
)

Again, nothing happens! This is because the `StreamingContext` must be started before the stream is processed by Spark. We can start data streaming using the [`start`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.start) method of the `StreamingContext` and stop it using the [`stop`](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext.stop) method. Let's run the stream for two minutes (120 seconds) and then stop:

In [ ]:
ssc.start()

time.sleep(120)

ssc.stop()